In [2]:
import numpy as np
import os
import shutil
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, LocallyConnected2D, Flatten, Dense, Dropout

2024-07-23 13:06:04.108635: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 13:06:04.154523: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 13:06:04.155783: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 13:06:04.996624: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
def get_deepface_model(input_shape = (200, 200, 3)):
    deepface = keras.Sequential(name = 'DeepFace')
    deepface.add(Convolution2D(32, (11, 11), activation = relu, name = 'C1', input_shape = input_shape))
    deepface.add(MaxPooling2D(pool_size = 3, strides = 2, padding = 'same', name = 'M2'))
    deepface.add(Convolution2D(16, (9, 9), activation = relu, name = 'C3'))
    deepface.add(LocallyConnected2D(16, (9, 9), activation = relu, name = 'L4'))
    deepface.add(LocallyConnected2D(16, (7, 7), strides = 2, activation = relu, name = 'L5'))
    deepface.add(LocallyConnected2D(16, (5, 5), activation = relu, name = 'L6'))
    deepface.add(Flatten(name = 'F0'))
    deepface.add(Dense(4096, activation = relu, name = 'F7'))
    deepface.add(Dropout(rate = 0.5, name = 'D0'))
    deepface.add(Dense(4038, activation = softmax, name = 'F8'))
    
    return deepface

In [4]:
def compile_model(model, **kwargs):
    model.compile(
        optimizer = kwargs['optimizer'],
        loss = kwargs['loss'],
        metrics = kwargs['metrics']
    )
    return

In [5]:
def create_callbacks(**kwargs):
    lr_reducer = keras.callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = kwargs['lr_factor'],
        patience = kwargs['lr_patience'],
        verbose = kwargs['lr_verbose'],
        mode = 'auto',
        min_delta = kwargs['lr_delta'],
        cooldown = kwargs['lr_cooldown'],
        min_lr = kwargs['lr_min']
    )
    early_stopper = keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = kwargs['stopper_delta'],
        patience = kwargs['stopper_patience'],
        verbose = kwargs['stopper_verbose'],
        mode = 'auto',
        baseline = None,
        restore_best_weights = False,
        start_from_epoch = kwargs['stopper_epoch']
    )
    checkpointer = keras.callbacks.ModelCheckpoint(
        filepath = kwargs['checkpoint_path'],
        monitor = 'val_loss',
        verbose = kwargs['checkpoint_verbose'],
        save_best_only = kwargs['checkpoint_best'],
        save_weights_only = kwargs['checkpoint_weights'],
        mode = 'auto',
        save_freq = kwargs['checkpoint_freq'],
        initial_value_thresold = kwargs['checkpoint_threshold']
    )
    
    return [lr_reducer, early_stopper, checkpointer]

In [6]:
def train_model(model, **kwargs):
    history = model.fit(
        x = kwargs['x_train'],
        y = kwargs['y_train'],
        epochs = kwargs['epochs'],
        batch_size = kwargs['batch_size'],
        callbacks = kwargs['callbacks'],
        validation_data = kwargs['val_data'],
        initial_epoch = kwargs['initial_epoch'],
        steps_per_epoch = kwargs['steps_per_epoch'],
        validation_steps = kwargs['validation_steps'],
        verbose = 1
    )
    
    return history

In [7]:
def get_label_mappings(train_dir):
    labels = ([' '.join(x.split('_')[0:-1]) for x in os.listdir(train_dir)])
    seen = set()
    unique_labels = [x for x in labels if not (x in seen or seen.add(x))]
    numeric_labels = [i for i, k in enumerate(unique_labels)]
    
#     table = tf.lookup.StaticHashTable(
#         tf.lookup.KeyValueTensorIniitalizer(tf.constant(unique_labels), tf.constant(numeric_labels)),
#         default_value = -1
#     )
    
#     return table
    return {unique_labels[i]:numeric_labels[i] for i in range(len(unique_labels))}

In [8]:
def write_label_mappings(mappings, path = './data/preprocessed/label_mappings.csv'):
    with open(path, 'w') as f:
        for i, key in enumerate(mappings.keys()):
            if i != len(mappings.keys()):
                line_sep = '\n'
            else:
                line_sep = ''
            f.write(f'{key},{mappings[key]}{line_sep}')

    return

In [9]:
def map_label(label, label_map, num_labels = 4038):
    numeric_label = label_map.lookup(label)
#     numeric_label = label_map[label]
    return tf.one_hot(indices = numeric_label, depth = num_labels)

In [10]:
@tf.function
def parse_image(file_path, label_map, num_labels = 4038):
    parts = tf.strings.split(file_path, os.path.sep)
#     label = tf.py_function(
#         map_label, inp = [' '.join(parts[-1].numpy().decode('utf-8').split(os.path.sep)[-1].split('_')[0:-1]),
#                            label_map, num_labels], Tout = [tf.float32])
    
#     label = map_label(tf.strings.join((parts[-1].numpy().decode('utf-8').split(os.path.sep)[-1].split('_')[0:-1])),
#                            label_map, num_labels = num_labels)
    
    label = parts[-1]
    label = tf.strings.split(label, os.path.sep)[-1]
    label = tf.strings.split(label, '_')[0:-1]
#     print(label)
    label = tf.strings.reduce_join(label, separator = ' ')
    label = map_label(label, label_map, num_labels = num_labels)
    
#     label = map_label(tf.strings.join((parts[-1].numpy().decode('utf-8').split(os.path.sep)[-1].split('_')[0:-1])),
#                            label_map, num_labels = num_labels)
    
    image = tf.io.read_file(file_path)
    image = tf.io.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    
    return image, label

In [11]:
training_image_dir = './data/preprocessed/padded/train'
validation_image_dir = './data/preprocessed/padded/validation'

if not os.path.exists(validation_image_dir):
    os.makedirs(validation_image_dir)
    
# If validation split has not occurred, create a validation split using labels with at least 2 images
# This ensure that the validation set data has representatives in the training set data with the same label
# In the LFW set, the training image set has 9525 images, and the number of labels with 2 images is 1184
# Thus, the validation holdout is 1184 / 9525 ~ 12.4% holdout
if len(os.listdir(training_image_dir)) == 9525 and len(os.listdir(validation_image_dir)) != 1184:
    validation_image_path_leafs = [x for x in os.listdir(training_image_dir) if '_0002' in x]
    
    for leaf in validation_image_path_leafs:
        shutil.move(os.path.join(training_image_dir, leaf), os.path.join(validation_image_dir, leaf))

In [12]:
labels = get_label_mappings(training_image_dir)
write_label_mappings(labels)

In [13]:
label_map = tf.lookup.StaticHashTable(
    tf.lookup.TextFileInitializer(
        './data/preprocessed/label_mappings.csv',
        key_dtype = tf.string, key_index = 0,
        value_dtype = tf.int64, value_index = 1,
        delimiter = ','
    ), default_value = -1
)

In [14]:
train_ds = tf.data.Dataset.list_files(os.path.join(training_image_dir, '*.jpg'))
train_ds = train_ds.map(lambda x: parse_image(x, label_map))
train_ds = train_ds.shuffle(8341).batch(1024).repeat()

In [15]:
val_ds = tf.data.Dataset.list_files(os.path.join(validation_image_dir, '*.jpg'))
val_ds = val_ds.map(lambda x: parse_image(x, label_map))
val_ds = val_ds.shuffle(1184).batch(1024).repeat()

In [16]:
deepface = get_deepface_model()

In [17]:
compile_params = {
    'optimizer': keras.optimizers.SGD(learning_rate = 0.01 * (1024 / 128), momentum = 0.9),
    'loss': keras.losses.categorical_crossentropy,
    'metrics': ['accuracy']
}
compile_model(deepface, **compile_params)

In [18]:
callback_params = {
    'lr_factor': 0.1,
    'lr_patience': 1,
    'lr_verbose': 1,
    'lr_delta': 0.0001,
    'lr_cooldown': 0,
    'lr_min': 0.0001,
    'stopper_delta': 0.001,
    'stopper_patience': 1,
    'stopper_verbose': 1,
    'stopper_epoch': 0,
    'checkpoint_path': './models/ckpt/deepface_{epoch}',
    'checkpoint_verbose': 1,
    'checkpoint_best': False,
    'checkpoint_weights': False,
    'checkpoint_freq': 'epoch',
    'checkpoint_threshold': None
}
callbacks = create_callbacks(**callback_params)

In [ ]:
training_params = {
    'x_train': train_ds,
    'y_train': None,
    'epochs': 1,
    'batch_size': 1024,
    'callbacks': callbacks,
    'val_data': val_ds,
    'initial_epoch': 0,
    'steps_per_epoch': (8341 // 1024) + 1,
    'validation_steps': (1184 // 1024) + 1
}

train_history = train_model(deepface, **training_params)